## Trees

*Sep 26, 2022*

There is two different metaphors used regularly to describe trees
1. **Recursive description (wooden trees):**
   - a **tree** has a root **label** and a list of **branches**
   - each branch is a **tree**
   - a tree with zero branches is called a **leaf**
  
2. **Relative Description** (family trees):
   - Each location in a tree is called a **node**
   - Each **node** has a **label** that can be any value
   - One node can be the **parent/child** of another


*People often refer to labels by their location: "each parent is the sum of its children"*

## Implementing the Tree Abstraction

for a tree:
```md
  3
 / \
1   2
   / \
  1   1
```
```py
>>> tree(3, [tree(1),
...          tree(2, [tree(1),
...                   tree(1)]))
[3., [1], [2, [1], [1]]]
```


notice that the first argument to tree is the label, and the second argument is a list of the branches

In [5]:
def tree(label, branches=[]):
    for branch in branches: # verifies the tree definition.
        assert is_tree(branch), 'branches must be trees'
    return [label] + list(branches) # list() ensures that if I pass in some other kind of sequence, it gets converted to a list before adding to another list.

def label(tree): # a selector
    return tree[0]

def branches(tree): # a selector
    return tree[1:]

def is_tree(obj):
    if type(obj) != list or len(obj) < 1:
        return False
    for branch in branches(obj): # recursively checks the branches of that tree. 
        if not is_tree(branch):
            return False
    return True

def is_leaf(obj):
    return not branches(obj) # empty branches

In [13]:
t = tree(1, [tree(5, [tree(7)]), tree(6)])

## Tree Processing

functions that take trees as inputs or return trees as outputs, are often tree recursive themselves. Lets look at an example.

In [17]:
def fib_tree(n):
    if n <= 1:
        return tree(n)
    else:
        left, right = fib_tree(n - 2), fib_tree(n - 1)
        return tree(label(left) + label(right), [left, right])

In [21]:
fib_tree(4)

[3, [1, [0], [1]], [2, [1], [1, [0], [1]]]]

In [22]:
def count_leaves(t): # my implementation
    if is_leaf(t):
        return 1
    else:
        total = 0
        for branch in branches(t):
            total += count_leaves(branch)
    return total

def count_leaves_john(t): # Prof. John implementation
    if is_leaf(t):
        return 1
    else:
        return sum([count_leaves(b) for b in branches(t)])

In [25]:
count_leaves(fib_tree(10))

89

### Discussion Question
implement `leaves` which returns a list of the leaf labels of a tree.

In [37]:
def leaves(t):
    if is_leaf(t):
        return [label(t)]
    else:
        return sum([leaves(b) for b in branches(t)], [])

leaves(fib_tree(5))

[1, 0, 1, 0, 1, 1, 0, 1]

### Creating Trees

a function that creates a tree from another tree is typically also recursive

In [39]:
def increment_leaves(t):
    """Return a tree like t but with leaf labels incremented"""
    if is_leaf(t):
        return tree(label(t) + 1)
    else:
        return tree(label(t), [increment_leaves(b) for b in branches(t)])

In [2]:
def increment(t):
    """Return a tree like t but with all labels incremented."""
    return tree(label(t) + 1, [increment(b) for b in branches(t)])
# Notice we don't need a base case since when we are at a leaf, it will have no branches, so the list comp. returns [], so increment will return tree(label(t) + 1, [])
# also notice that we won't make any recursive calls when we are at a leaf (list comp. won't loop.)

### Printing Trees

In [63]:
def print_tree(t, indent=0):
    print(' ' * indent + str(label(t)))
    for b in branches(t):
        print_tree(b, indent + 3) # indentation by 3 to make it more clear


# note: indentation level of a label, corresponds to it's depth in the tree.

In [65]:
print_tree(fib_tree(4))

3
   1
      0
      1
   2
      1
      1
         0
         1


### Example: Summing Paths
summing all the labels along a path from the root to a leaf of a tree, then printing out that sum.

In [68]:
def print_sums(t):
    def sums(t, sum_so_far):
        sum_so_far += label(t)
        if is_leaf(t):
            print(sum_so_far)
        else:
            for b in branches(t):
                print_sums(b, sum_so_far)
    return sums(t, 0)

### Example: Counting Paths

In [69]:
def count_paths(t, total):
    """Return the number of paths from the root to any node in t for which the labels along the path sum to total"""
    if label(t) == total:
        found = 1
    else:
        found = 0
    return found + sum([count_paths(b, total - label(t)) for b in branches(t)])